In [2]:
# Imports for visualization
import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
import pyautogui

In [3]:
def getRandomFEN():
    fen_chars = list('1KQRBNPkqrbnp')
    pieces = np.random.choice(fen_chars, 64)
    fen = '/'.join([''.join(pieces[i*8:(i+1)*8]) for i in range(8)])
    # can append ' w' or ' b' for white/black to play, defaults to white
    return fen

fen = getRandomFEN()
print(fen + ' w KQkq - 0 1')
print(fen.replace('/','-'))

QrKQqBbn/QKr1rRk1/BPnPbNpr/r1kKQrqq/nRBKrNBb/1pkpKQQR/bPpB1pbP/r1bppqkN w KQkq - 0 1
QrKQqBbn-QKr1rRk1-BPnPbNpr-r1kKQrqq-nRBKrNBb-1pkpKQQR-bPpB1pbP-r1bppqkN


In [4]:
url = "http://en.lichess.org/editor/%s" % getRandomFEN()
print(url)

http://en.lichess.org/editor/rNrNbK1P/NqbPQRN1/QQP1PNBb/RNbrkNQn/BBnqnPNp/KkNkBbrq/nRRbpb1b/NPRQBQ1r


In [5]:
def take_screenshot():
    myScreenshot = pyautogui.screenshot(region=(500,290, 500, 500))
    myScreenshot.save(r'screenshot1.png')
    return r'screenshot1.png'

In [6]:
import tkinter as tk
from tkinter import filedialog



In [7]:
def display_image(img):
    #imS = cv2.resize(img, (1000, 700))  # Resize image
    imS = cv2.resize(img, (800, 800))
    cv2.imshow("image", imS)
    cv2.waitKey(0)

    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

def takeScreenshot ():
    myScreenshot = pyautogui.screenshot()
    file_path = filedialog.asksaveasfilename(defaultextension='.png')
    myScreenshot.save(file_path)

def checkMatch(lineset):
    """Checks whether there exists 7 lines of consistent increasing order in set of lines"""
    linediff = np.diff(lineset)
    x = 0
    cnt = 0
    for line in linediff:
        # Within 5 px of the other (allowing for minor image errors)
        if np.abs(line - x) < 5:
            cnt += 1
        else:
            cnt = 0
            x = line
    return cnt == 5

#Remove lines which are very close to each other so that the board detection will be easier. 
def filter_lines(lines, linediff):
    indices_to_delete = []
    for index, diff in enumerate(linediff): 
        if(diff[0] <= 10):
            indices_to_delete.append(index)
    return np.delete(lines, indices_to_delete,0)

def get_chess_lines(lines, linediff):
    indices_to_delete = []
    #find most occuring difference which is probably the width of the chess square. 
    m = stats.mode(linediff)
    mode = m[0][0][0]
    for index, diff in enumerate(linediff): 
        if(abs(diff[0] - mode) >5):
            print(diff[0])
            indices_to_delete.append(index+1)
    return np.delete(lines, indices_to_delete,0)

#Get equations of lines:
def getIntersection(rho1,theta1,rho2, theta2):
    A = np.array([
        [np.cos(theta1), np.sin(theta1)],
        [np.cos(theta2), np.sin(theta2)]
    ])
    b = np.array([[rho1], [rho2]])
    x0, y0 = np.linalg.solve(A, b)
    x0, y0 = int(np.round(x0)), int(np.round(y0))
    return [[x0, y0]]
    
def get_square_coordinates(points):
    squares = []
    for i in range(0, 8):
        for j in range(0,8):
            squares.append([(i,j), (i, j + 1), (i + 1, j), (i + 1, j + 1)])
    return squares

def get_ith_square(i, points, squares):
    square = squares[i-1]
    return [points[square[0][0]][square[0][1]], points[square[1][0]][square[1][1]],
            points[square[2][0]][square[2][1]],points[square[3][0]][square[3][1]]]

def decode(s):
    if s == 'wr':
        return 0
    if s == 'wh':
        return 1
    if s == 'wb':
        return 2
    if s == 'wq':
        return 3
    if s == 'wk':
        return 4
    if s == 'wp':
        return 5
    if s == 'br':
        return 6
    if s == 'bh':
        return 7
    if s == 'bb':
        return 8
    if s == 'bq':
        return 9
    if s == 'bk':
        return 10
    if s == 'bp':
        return 11
    else:
        return 12

In [8]:
l1 = []
l2 = []
for i in range(10):
    url = "http://en.lichess.org/editor/%s" % getRandomFEN()
    print(url)
    root= tk.Tk()

    canvas1 = tk.Canvas(root, width = 300, height = 300)
    canvas1.pack()

    def takeScreenshot ():
        myScreenshot = pyautogui.screenshot()
        file_path = filedialog.asksaveasfilename(defaultextension='.png')
        myScreenshot.save(file_path)

    myButton = tk.Button(text='Take Screenshot', command=take_screenshot, bg='green',fg='white',font= 10)
    canvas1.create_window(150, 150, window=myButton)

    root.mainloop()

    #Load image and convert to grayscale
    image_path = 'screenshot1.png'
    img = cv2.imread(image_path)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    #Perform Hough transform
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray,50,150,apertureSize = 3)
    lines = cv2.HoughLines(edges,1,np.pi/180,200)

    vertical_lines = []
    horizontal_lines = []
    #Display all lines
    for line in lines:
        rho = line[0][0]
        theta = line[0][1]
        if(theta == 0):
            vertical_lines.append(line)



    #Display all lines
    for line in lines:
        rho = line[0][0]
        theta = line[0][1]
        if theta != 0.0:        
            horizontal_lines.append(line)

    #Convert to numpy arrays. 
    horizontal_lines_np = np.concatenate( horizontal_lines, axis=0 )
    vertical_lines_np = np.concatenate( vertical_lines, axis=0 )
    horizontal_lines_np = np.sort(horizontal_lines_np, axis = 0)
    vertical_lines_np = np.sort(vertical_lines_np, axis = 0)

    linediff_hor = np.diff(horizontal_lines_np, axis = 0)
    linediff_ver = np.diff(vertical_lines_np, axis = 0)

    filtered_horizontal_lines = filter_lines(horizontal_lines_np, linediff_hor)
    filtered_vertical_lines = filter_lines(vertical_lines_np, linediff_ver)

    #update line differences. 

    linediff_hor = np.diff(filtered_horizontal_lines, axis = 0)
    linediff_ver = np.diff(filtered_vertical_lines, axis = 0)

    chess_lines_hor = get_chess_lines(filtered_horizontal_lines, linediff_hor)
    chess_lines_ver = get_chess_lines(filtered_vertical_lines, linediff_ver)

    #Get all intersection points of horizontal and vertical lines to get the corners of the chess squares to extract patches. 

    points = []
    for i in chess_lines_hor:
        for j in chess_lines_ver:
            rho_hor = i[0]
            theta_hor = i[1]
            rho_ver = j[0]
            theta_ver = j[1]
            points.append(getIntersection(rho_hor, theta_hor, rho_ver, theta_ver)[0])

    # Make empty black image
    img = cv2.imread(image_path)
    for point in points:
        cv2.circle(img, (point[0],point[1]), 5, [0,0,255], thickness=2, lineType=8, shift=0)
    #image[10,5]=[0,0,255]

    display_image(img)
    # # Make one pixel red

    # # Save
    # cv2.imwrite("result.png",image)

    # create a 2D list for easier point arrangement
    new_points = []

    if(len(points) != 81) :
        print("Board not detected! Try again!")
        continue
    else:
        for x in range(0,81):
            if(x % 9) == 0:
                new_points.append([])
            new_points[int(x/9)].append(points[x])

    squares = get_square_coordinates(new_points)
    img = cv2.imread(image_path)

    for square in range(1,65):
        corners = get_ith_square(square, new_points, squares)
        img_patch = img[corners[0][0] : corners[3][0], corners[0][1] : corners[3][1]]
        img_patch_short = cv2.resize(cv2.cvtColor(img_patch, cv2.COLOR_BGR2GRAY),(28,28))
        display_image(img_patch_short)
        l1.append(img_patch_short)
        l2.append(decode(input()))


http://en.lichess.org/editor/rNbbqrnK/rNrpqnpq/bnbRQNqr/RRKNKPqB/1nNPqbRb/bNbRqPp1/NrpQQQKb/PpNrrqrR
br
br
bb
wr
n
bb
wh
wp
wh
wh
bh
wr
bh
wh
br
bp
bb
br
bb
wk
wh
bb
bp
wh
bb
bp
w
wh
wp
wr
wq
br
bq
bq
wq
wk
bq
bq
wq
br
br
bh
wh
wp
bb
wp
wk
bk
bh
bp
wk
wk
bh
bp
wk
br
wk
bq
br
wb
bb
n
bb
wr
http://en.lichess.org/editor/rQNkNbbk/BK11KNnr/q1rkKqpK/brBQrPPR/pbQQN1pP/1NQrbPpQ/Pb1q1KBb/BPBbrknr
49.0
13.0
Board not detected! Try again!
http://en.lichess.org/editor/kPPqqbkR/Rp1NkNPB/NNQrpnbN/b1qb1bQk/nrpNprkK/rNRnkpqq/rkqNnRkN/Kr1NkPQb
bk
wr
wh
bb
bh
br
br
wk
wp
bp
wh
n
br
wh
bk
br
wp
n
wq
bq
bp
wr
bq
n
bq
wh
br
bb
wh
bh
wh
wh
bq
bk
bp
n
bp
bk
bh
bk
bb
wh
bh
bb
br
bp
wr
wp
bk
wp
bb
wq
bk
bq
bk
wq
wr
wb
wh
bk
wk
bq
wh
bb
http://en.lichess.org/editor/QkKknrN1/k1PkqBNr/qkbPRbNn/qrrpRnRp/1NRkrnP1/P1bbBkpb/QbkRQrqR/nR1pnpKR
wq
bk
bq
bq
n
wp
wq
bh
bk
n
bk
br
wh
n
bb
wr
wk
wp
bb
br
wr
bb
bk
n
bk
bk
wp
bp
bk
bb
wr
bp
bh
bq
wr
wr
br
wb
wq
bh
br
wb
bb
bh
bh
bk
br
bp
wb
wh
xwh
wr
wp
bp
bq
wk
n
br
bh
bp
n


error: OpenCV(3.4.1) C:\Miniconda3\conda-bld\opencv-suite_1533128839831\work\modules\imgproc\src\resize.cpp:4044: error: (-215) ssize.width > 0 && ssize.height > 0 in function cv::resize


AttributeError: 'list' object has no attribute 'shape'

In [12]:
d1 = np.dstack(l1)
d2 = np.array(l2)

d1.shape #(5,3,5)
d2.shape

(512,)

In [13]:
from numpy import save
save('dataset_x.npy', d1)
save('dataset_y.npy',d2)

In [ ]:
l1